In [1]:
import hddCRP.simulations
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx

In [ ]:
# session_lengths = [50,55,53,50,50, 50,50,50,50,50,50,50,50,50,50] # number of observations (actions) per session
# session_labels = ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']

session_lengths = [4,3]#[10, 15]
session_labels  = ['A', 'B']

action_labels = ['L', 'R', 'S']

depth = 3; # look 2 actions in the past
alphas = [2,3,3] # concentration parameter
between_session_time_constants = np.array([[3, 1],
                                            [1, 3]]) # units = sessions
within_session_time_constant = 20 # units = actions

seqs, connection_data = hddCRP.simulations.simulate_sequential_hddCRP(session_lengths, session_labels, action_labels, depth, alphas, between_session_time_constants, within_session_time_constant)
print(seqs[:2])


In [ ]:
num_sessions_to_plot = min(3, len(session_lengths));

if(np.sum(session_lengths[:num_sessions_to_plot]) < 50):
    ax = plt.gca()
    hddCRP.simulations.make_graph_plot(connection_data, ax, num_sessions=num_sessions_to_plot)
    plt.show()
else:
    print("not plotting graph: too many nodes")


In [ ]:
model = hddCRP.simulations.create_hddCRPModel_from_simulated_sequential_hddCRP(seqs, connection_data)

In [ ]:
model.run_gibbs_sweep()
#model._gibbs_sample_single_node(0, 0)

In [ ]:
print(connection_data["C_ctx"])

In [ ]:
print(model._C_predecessors[0])
print(model._C_predecessors[1])
print(model._C_predecessors[2])

In [ ]:
print(model._C_tables)

In [ ]:
print(model._C_table_values)

In [ ]:
make_graph_plot

In [ ]:
print(model._C_num_labeled_in_table)

In [ ]:
num_sessions_to_plot = min(3, len(session_lengths));

connection_data2 = {"C_y" : model._C_y, "C_ctx" : connection_data["C_ctx"], "session_lengths" : connection_data["session_lengths"]}

if(np.sum(session_lengths[:num_sessions_to_plot]) < 50):
    ax = plt.gca()
    hddCRP.simulations.make_graph_plot(connection_data2, ax, num_sessions=num_sessions_to_plot)
    plt.show()
else:
    print("not plotting graph: too many nodes")